In [3]:
#Setup 
import requests
import json
import os
from time import sleep
from datetime import date

WIKI_URL = "https://en.wikipedia.org"
API_ENDPOINT = WIKI_URL + "/w/api.php"
LINK_LIMIT = 100
DATA_FILE = 'networkdata3.json'

In [4]:
try:
    with open('/home/teijehidde/Documents/Git Blog and Coding/Project one (wikipedia SNA)/Code/' + DATA_FILE) as json_file:
        global network_data
        network_data = json.load(json_file)
    
except IOError:
    print("Error: Could not find " + DATA_FILE + ". Please check if file is present in directory, or change DATA_FILE value.")
    
else: 
    print("The file " + DATA_FILE + " found, succesfully loaded.")
    print("Network data accessed through " + API_ENDPOINT + ".")

The file networkdata3.json found, succesfully loaded.
Network data accessed through https://en.wikipedia.org/w/api.php.


In [5]:
network_data.keys()

dict_keys(['Zevenaar', 'Duiven', 'Cambridge', 'Albert Einstein'])

In [8]:
print("Current networks in " + DATA_FILE + ":")
print("  ")
    
for item in network_data.keys():
    if network_data[item]['ego'] == 1:
        print(item)
            

Current networks in networkdata3.json:
  
Zevenaar
Duiven
Cambridge


KeyError: 'ego'